In [3]:
import numpy as np
import copy, math
import matplotlib.pyplot as plt
from sklearn.feature_selection import SequentialFeatureSelector
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import logging

In [4]:
training_set = 800

def load_data():
    data = np.loadtxt("../data/Cardiovascular_Disease_Dataset.csv", delimiter=',', skiprows=1)

    X = data[:,1:-1]
    y = data[:,-1]
    return X, y

x_tmp, y_tmp = load_data()
X_train = x_tmp[:training_set]
X_test = x_tmp[training_set:]
y_train = np.expand_dims(y_tmp[:training_set], axis=1)
y_test = np.expand_dims(y_tmp[training_set:], axis=1)

norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X_train)  # learns mean, variance
X_norm = norm_l(X_train)
X_test_norm = norm_l(X_test)

In [7]:
model = Sequential(
    [
        tf.keras.Input(shape=(12,)),
        Dense(12, activation='relu', name = 'layer1'),
        Dense(24, activation='relu', name = 'layer2'),
        Dense(1, activation='sigmoid', name = 'layer3')
     ]
)

model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.01),
)

res = model.fit(
    X_norm,
    y_train,            
    epochs=100,
)

print(res)


Epoch 1/100
25/25 [==============================] - 0s 659us/step - loss: 0.5227
Epoch 2/100
25/25 [==============================] - 0s 551us/step - loss: 0.1940
Epoch 3/100
25/25 [==============================] - 0s 529us/step - loss: 0.1174
Epoch 4/100
25/25 [==============================] - 0s 890us/step - loss: 0.0861
Epoch 5/100
25/25 [==============================] - 0s 540us/step - loss: 0.0685
Epoch 6/100
25/25 [==============================] - 0s 497us/step - loss: 0.0590
Epoch 7/100
25/25 [==============================] - 0s 504us/step - loss: 0.0526
Epoch 8/100
25/25 [==============================] - 0s 489us/step - loss: 0.0403
Epoch 9/100
25/25 [==============================] - 0s 478us/step - loss: 0.0386
Epoch 10/100
25/25 [==============================] - 0s 480us/step - loss: 0.0334
Epoch 11/100
25/25 [==============================] - 0s 481us/step - loss: 0.0307
Epoch 12/100
25/25 [==============================] - 0s 491us/step - loss: 0.0289
Epoch 13/100


In [11]:
predictions = model.predict(X_test_norm)
predictions.squeeze()

false_positive = 0
false_negative = 0
for i, p in enumerate(predictions):
    if (p > 0.5):
        p = 1
    else:
        p = 0

    if (p == 1 and y_test[i] == 0):
        false_positive += 1

    if (p == 0 and y_test[i] == 1):
        false_negative += 1
    print("Result for normalized:", p, "vs actual:", y_train[i])

m = len(predictions)
print("Total false positives:", false_positive)
print("Total false negatives:", false_negative)
print("Total predictions:", m)
print("Error rate:", (false_negative + false_positive) / m * 100)

7/7 [==============================] - 0s 636us/step
Result for normalized: 1 vs actual: [1.]
Result for normalized: 0 vs actual: [0.]
Result for normalized: 0 vs actual: [0.]
Result for normalized: 1 vs actual: [1.]
Result for normalized: 0 vs actual: [1.]
Result for normalized: 0 vs actual: [1.]
Result for normalized: 1 vs actual: [1.]
Result for normalized: 1 vs actual: [0.]
Result for normalized: 1 vs actual: [0.]
Result for normalized: 1 vs actual: [1.]
Result for normalized: 1 vs actual: [1.]
Result for normalized: 0 vs actual: [1.]
Result for normalized: 1 vs actual: [0.]
Result for normalized: 0 vs actual: [0.]
Result for normalized: 1 vs actual: [0.]
Result for normalized: 1 vs actual: [1.]
Result for normalized: 0 vs actual: [1.]
Result for normalized: 1 vs actual: [0.]
Result for normalized: 1 vs actual: [1.]
Result for normalized: 0 vs actual: [0.]
Result for normalized: 0 vs actual: [1.]
Result for normalized: 0 vs actual: [1.]
Result for normalized: 1 vs actual: [0.]
Resu